In [3]:
from datasets import load_dataset

# Loading the main split of the dataset
ds_main = load_dataset("openai/gsm8k", "main")

/opt/anaconda3/envs/agential/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 1319/1319 [00:00<00:00, 445111.19 examples/s]


In [4]:

root_dir = "output"
method_name = "standard"
benchmark = "gsm8k"

model = "gpt-3.5-turbo"
eval_model = "gpt-4o-mini"
seed = 42
num_retries = 1
warming = [0.0]

In [5]:
from agential.core.llm import LLM
import os
llm = LLM(
        model, 
        organization=os.getenv("OPENAI_ORGANIZATION"), 
        temperature=0,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        seed=seed
)
eval_llm = LLM(
    eval_model,
    organization=os.getenv("OPENAI_ORGANIZATION"),
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    seed=seed
)

In [8]:
import wandb
wandb.login()
from agential.prompting.standard.prompting import Standard
method = Standard(
        llm=llm,
        benchmark=benchmark,
)
run = wandb.init(
    project=benchmark, 
    entity="agential",
    config={
        "model": model,
        "eval_model": eval_model,
        "seed": seed,
        "num_retries": num_retries,
        "warming": warming,
    },
    group=method_name,
    tags=[f"method={method_name}", f"model={model}", f"eval_model={eval_model}", f"seed={seed}", f"num_retries={num_retries}", f"warming={warming}"],
)

In [12]:
ds_main['test'][0]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [13]:
question = ds_main['test'][0]["question"]
answer = ds_main['test'][0]["answer"]


question , answer

("Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18')

In [14]:
# Inference.
out = method.generate(
    question=question,
    key=answer,
    num_retries=num_retries,
    warming=warming
)


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - Traceback (most recent call last):
  File "/opt/anaconda3/envs/agential/lib/python3.11/site-packages/litellm/llms/openai.py", line 1026, in completion
    raise e
  File "/opt/anaconda3/envs/agential/lib/python3.11/site-packages/litellm/llms/openai.py", line 921, in completion
    openai_client = self._get_openai_client(
                    ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/agential/lib/python3.11/site-packages/litellm/llms/openai.py", line 743, in _get_openai_client
    _new_client = openai(
                  ^^^^^^^
  File "/opt/anaconda3/envs/agential/lib/python3.11/site-packages/openai/_client.py", line 104, in __init__
    raise openaiError(
openai.openaiError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
